# BME680 Temperature, Pressure, Humidity & Gas Sensor
This code is based on https://github.com/pimoroni/bme680-python


In [1]:
import bme680
import constants
import time
import threading

In [2]:
from smbus import SMBus
i2cbus = SMBus(0)

In [30]:
print("Display initial temperature, pressure, humidity, and gas.")

try:
    sensor = bme680.BME680(bme680.I2C_ADDR_PRIMARY,i2cbus)
except (RuntimeError, IOError):
    sensor = bme680.BME680(bme680.I2C_ADDR_SECONDARY,i2cbus)

# These calibration data can safely be commented
# out, if desired.

#print('Calibration data:')
#for name in dir(sensor.calibration_data):
#
#    if not name.startswith('_'):
#        value = getattr(sensor.calibration_data, name)
#
#        if isinstance(value, int):
#            print('{}: {}'.format(name, value))

# These oversampling settings can be tweaked to
# change the balance between accuracy and noise in
# the data.

sensor.set_humidity_oversample(bme680.OS_2X)
sensor.set_pressure_oversample(bme680.OS_4X)
sensor.set_temperature_oversample(bme680.OS_8X)
sensor.set_filter(bme680.FILTER_SIZE_3)
sensor.set_gas_status(bme680.ENABLE_GAS_MEAS)

#Print each initial reading
for name in dir(sensor.data):
    value = getattr(sensor.data, name)

    if not name.startswith('_'):
        print('{}: {}'.format(name, value))

# Up to 10 heater profiles can be configured, each
# with their own temperature and duration.
# sensor.set_gas_heater_profile(200, 150, nb_profile=1)
# sensor.select_gas_heater_profile(1)
sensor.set_gas_heater_temperature(320)
sensor.set_gas_heater_duration(150)
sensor.select_gas_heater_profile(0)

Display initial temperature, pressure, humidity, and gas.
gas_index: 0
gas_resistance: 9230769.762871865
heat_stable: False
humidity: 48.233
meas_index: 0
pressure: 1028.92
status: 32
temperature: 21.72


In [31]:
#Define a worker thread to poll the sensor
continue_worker_thread = True
def work():
    while continue_worker_thread == True:
        if sensor.get_sensor_data():
            output = '{0:.2f} C,{1:.2f} hPa,{2:.2f} %RH'.format(
                sensor.data.temperature,
                sensor.data.pressure,
                sensor.data.humidity)

            if sensor.data.heat_stable:
                print('{0},{1} Ohms'.format(
                    output,
                    sensor.data.gas_resistance))

            else:
                print(output)

        time.sleep(1)

In [32]:
#Start polling for values
thread = threading.Thread(target=work)
thread.start()

21.72 C,1028.93 hPa,48.22 %RH
21.75 C,1028.93 hPa,48.20 %RH,12123.344766153705 Ohms
21.82 C,1028.94 hPa,48.10 %RH,16835.096996726825 Ohms


In [33]:
#Stop the worker thread
continue_worker_thread = False

In [ ]:
from smbus import SMBus
i2cbus = SMBus(0)
print("""Estimate indoor air quality.
Runs the sensor for a burn-in period, then uses a
combination of relative humidity and gas resistance
to estimate indoor air quality as a percentage.
Press Ctrl+C to exit!
""")

try:
    sensor = BME680(I2C_ADDR_PRIMARY,i2cbus)
except (RuntimeError, IOError):
    sensor = BME680(I2C_ADDR_SECONDARY,i2cbus)

# These oversampling settings can be tweaked to
# change the balance between accuracy and noise in
# the data.

sensor.set_humidity_oversample(OS_2X)
sensor.set_pressure_oversample(OS_4X)
sensor.set_temperature_oversample(OS_8X)
sensor.set_filter(FILTER_SIZE_3)
sensor.set_gas_status(ENABLE_GAS_MEAS)

sensor.set_gas_heater_temperature(320)
sensor.set_gas_heater_duration(150)
sensor.select_gas_heater_profile(0)

# start_time and curr_time ensure that the
# burn_in_time (in seconds) is kept track of.

start_time = time.time()
curr_time = time.time()
burn_in_time = 300

burn_in_data = []

try:
    # Collect gas resistance burn-in values, then use the average
    # of the last 50 values to set the upper limit for calculating
    # gas_baseline.
    print('Collecting gas resistance burn-in data for 5 mins\n')
    while curr_time - start_time < burn_in_time:
        curr_time = time.time()
        if sensor.get_sensor_data() and sensor.data.heat_stable:
            gas = sensor.data.gas_resistance
            burn_in_data.append(gas)
            print('Gas: {0} Ohms'.format(gas))
            time.sleep(1)

    gas_baseline = sum(burn_in_data[-50:]) / 50.0

    # Set the humidity baseline to 40%, an optimal indoor humidity.
    hum_baseline = 40.0

    # This sets the balance between humidity and gas reading in the
    # calculation of air_quality_score (25:75, humidity:gas)
    hum_weighting = 0.25

    print('Gas baseline: {0} Ohms, humidity baseline: {1:.2f} %RH\n'.format(
        gas_baseline,
        hum_baseline))

    while True:
        if sensor.get_sensor_data() and sensor.data.heat_stable:
            gas = sensor.data.gas_resistance
            gas_offset = gas_baseline - gas

            hum = sensor.data.humidity
            hum_offset = hum - hum_baseline

            # Calculate hum_score as the distance from the hum_baseline.
            if hum_offset > 0:
                hum_score = (100 - hum_baseline - hum_offset)
                hum_score /= (100 - hum_baseline)
                hum_score *= (hum_weighting * 100)

            else:
                hum_score = (hum_baseline + hum_offset)
                hum_score /= hum_baseline
                hum_score *= (hum_weighting * 100)

            # Calculate gas_score as the distance from the gas_baseline.
            if gas_offset > 0:
                gas_score = (gas / gas_baseline)
                gas_score *= (100 - (hum_weighting * 100))

            else:
                gas_score = 100 - (hum_weighting * 100)

            # Calculate air_quality_score.
            air_quality_score = hum_score + gas_score

            print('Gas: {0:.2f} Ohms,humidity: {1:.2f} %RH,air quality: {2:.2f},hum_score: {3:.2f},gas_score: {4:.2f}'.format(
                gas,
                hum,
                air_quality_score,
                hum_score,
                gas_score))

            time.sleep(1)

except KeyboardInterrupt:
    pass